In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import re
import random

USER_REVIEWS_CACHE_FILE = "user_reviews_cache_selenium.json"

def load_cache(cache_file):
    try:
        with open(cache_file, encoding="utf-8") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return {}

def save_cache(cache_file, data):
    with open(cache_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def scrape_user_ids_selenium(film_id):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Ohne GUI
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = ChromeService()  # Pfad zum Chromedriver, falls nötig, z.B. ChromeService(executable_path="pfad/zum/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)

    base_url = f"https://www.filmstarts.de/kritiken/{film_id}/userkritiken/"
    user_ids = []

    cache = load_cache(USER_REVIEWS_CACHE_FILE)
    if film_id in cache:
        print(f"User-IDs für Film {film_id} aus Cache geladen.")
        return cache[film_id]

    page = 1
    print(f"Starte Scraping für Film {film_id}...")

    try:
        while True:
            url = base_url if page == 1 else f"{base_url}?page={page}"
            print(f"Scraping Seite {page} - {url}")

            driver.get(url)

            # Warte, bis die Userinfos geladen sind oder Timeout nach 10 Sekunden
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.review-card-user-infos"))
                )
            except TimeoutException:
                print("  Timeout beim Warten auf User-Infos. Eventuell keine User auf dieser Seite.")
                # Falls keine Userinfos, prüfen ob nächste Seite existiert oder abbrechen
                pass

            # User-Infos finden
            user_infos = driver.find_elements(By.CSS_SELECTOR, "div.review-card-user-infos")
            if not user_infos:
                print("  Keine Userinfos auf dieser Seite gefunden.")

            for info in user_infos:
                try:
                    # User-ID aus href extrahieren
                    a_tag = info.find_element(By.CSS_SELECTOR, "a[href*='/mitglieder/']")
                    href = a_tag.get_attribute("href")
                    user_id_match = re.search(r"/mitglieder/([^/]+)/", href)
                    if not user_id_match:
                        continue
                    user_id = user_id_match.group(1)

                    # Anzahl Kritiken extrahieren
                    kritik_link = info.find_element(By.CSS_SELECTOR, "a[href*='/kritik/filme/']")
                    kritik_text = kritik_link.text
                    anzahl_match = re.search(r"(\d+)", kritik_text)
                    if anzahl_match:
                        anzahl_kritiken = int(anzahl_match.group(1))
                        if anzahl_kritiken >= 10 and user_id not in user_ids:
                            user_ids.append(user_id)
                except NoSuchElementException:
                    continue

            # Pagination prüfen: Gibt es einen aktiven „Nächste“-Link?
            try:
                # Aktiver Link „Nächste“ ist ein <a> mit class button-right, kein disabled span
                next_button = driver.find_element(By.CSS_SELECTOR, "nav.pagination a.button-right")
                # Wenn gefunden, nächstes page laden
                page += 1
                # Kurze Pause, um nicht zu schnell zu scrapen
                time.sleep(random.uniform(1, 2))
            except NoSuchElementException:
                print("  Keine weitere Seite vorhanden. Beende Scraping.")
                break

    finally:
        driver.quit()

    # Cache speichern
    cache[film_id] = user_ids
    save_cache(USER_REVIEWS_CACHE_FILE, cache)

    return user_ids

In [3]:
ids = scrape_user_ids_selenium("257724")  # Beispiel-Kritiken-ID
print(ids)


Starte Scraping für Film 257724...
Scraping Seite 1 - https://www.filmstarts.de/kritiken/257724/userkritiken/
Scraping Seite 2 - https://www.filmstarts.de/kritiken/257724/userkritiken/?page=2
  Keine weitere Seite vorhanden. Beende Scraping.
['Z20160503220447373821382', 'Z20160604100830347223266', 'Z20230420134156746954294', 'Z20200422164436580136284', 'Z20200827105330244019355']


In [ ]:
import csv
import json
import re
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import sys

csv.field_size_limit(10_000_000)  # 10 MB pro Feld, reicht für sehr große Inhalte


MAX_USERS = 10000
USER_REVIEWS_CACHE_FILE = "user_reviews_cache_selenium.json"

def load_cache(cache_file):
    try:
        with open(cache_file, encoding="utf-8") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return {}

def save_cache(cache_file, data):
    with open(cache_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def film_id_from_url(url):
    # Extrahiere die Film-ID aus der URL (z.B. https://.../kritiken/10053.html -> 10053)
    match = re.search(r"/kritiken/(\d+)", url)
    if match:
        return match.group(1)
    return None

def scrape_user_ids_selenium(film_id, max_users_per_film=10000):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = ChromeService()  # ggf. ChromeDriver-Pfad hier angeben
    driver = webdriver.Chrome(service=service, options=options)

    base_url = f"https://www.filmstarts.de/kritiken/{film_id}/userkritiken/"
    user_ids = []

    page = 1
    print(f"Starte Scraping für Film {film_id}...")

    try:
        while True:
            url = base_url if page == 1 else f"{base_url}?page={page}"
            print(f"Scraping Seite {page} - {url}")

            driver.get(url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.review-card-user-infos"))
                )
            except TimeoutException:
                print("   Timeout beim Warten auf User-Infos. Eventuell keine User auf dieser Seite.")
                # Wenn Timeout -> abbrechen, keine weiteren Seiten
                break

            user_infos = driver.find_elements(By.CSS_SELECTOR, "div.review-card-user-infos")
            if not user_infos:
                print("   Keine Userinfos auf dieser Seite gefunden.")
                break

            for info in user_infos:
                try:
                    a_tag = info.find_element(By.CSS_SELECTOR, "a[href*='/mitglieder/']")
                    href = a_tag.get_attribute("href")
                    user_id_match = re.search(r"/mitglieder/([^/]+)/", href)
                    if not user_id_match:
                        continue
                    user_id = user_id_match.group(1)

                    kritik_link = info.find_element(By.CSS_SELECTOR, "a[href*='/kritik/filme/']")
                    kritik_text = kritik_link.text
                    anzahl_match = re.search(r"(\d+)", kritik_text)
                    if anzahl_match:
                        anzahl_kritiken = int(anzahl_match.group(1))
                        if anzahl_kritiken >= 10 and user_id not in user_ids:
                            user_ids.append(user_id)
                            if len(user_ids) >= max_users_per_film:
                                print(f"Maximale Anzahl User ({max_users_per_film}) für Film erreicht.")
                                return user_ids
                except NoSuchElementException:
                    continue

            # Prüfen ob es eine nächste Seite gibt
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "nav.pagination a.button-right")
                page += 1
                time.sleep(random.uniform(1, 2))
            except NoSuchElementException:
                print("   Keine weitere Seite vorhanden. Beende Scraping.")
                break
    finally:
        driver.quit()

    return user_ids

def scrape_all_films(csv_file_path):
    cache = load_cache(USER_REVIEWS_CACHE_FILE)

    all_user_ids = set()
    # Alle User aus Cache global sammeln
    for user_list in cache.values():
        all_user_ids.update(user_list)

    total_users = len(all_user_ids)
    print(f"Bereits im Cache gespeicherte User: {total_users}")

    with open(csv_file_path, encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            url = row["URL"]
            film_id = film_id_from_url(url)

            if not film_id:
                print(f"Keine Film-ID aus URL {url} gefunden, überspringe...")
                continue

            if film_id in cache:
                print(f"Film {film_id} User-IDs aus Cache geladen.")
                user_ids = cache[film_id]
            else:
                print(f"Scraping Film-ID {film_id} ...")
                user_ids = scrape_user_ids_selenium(film_id)
                cache[film_id] = user_ids
                # Cache nach jedem Film speichern
                save_cache(USER_REVIEWS_CACHE_FILE, cache) 


            unique_new_user_ids = [uid for uid in user_ids if uid not in all_user_ids]

            remaining_slots = MAX_USERS - total_users
            if remaining_slots <= 0:
                print("Maximale Anzahl von Usern erreicht, Stop.")
                break

            if len(unique_new_user_ids) > remaining_slots:
                unique_new_user_ids = unique_new_user_ids[:remaining_slots]

            all_user_ids.update(unique_new_user_ids)
            total_users += len(unique_new_user_ids)
            print(f"Gesamt User bisher: {total_users}")

            if total_users >= MAX_USERS:
                print("Maximale Anzahl von Usern erreicht, Stop.")
                break

    # Dies wird nur ausgeführt, wenn die Schleife natürlich endet (nicht durch 'break')
    # oder wenn der `MAX_USERS` Wert nicht erreicht wird, bevor alle Filme verarbeitet wurden.
    # Da wir jetzt innerhalb der Schleife speichern, ist diese Zeile optional,
    # kann aber beibehalten werden, um sicherzustellen, dass der letzte Zustand immer gespeichert wird.
    save_cache(USER_REVIEWS_CACHE_FILE, cache) 

    user_list = list(all_user_ids)
    save_cache("all_user_ids.json", user_list)
    print(f"Gesamt {len(user_list)} einzigartige User-IDs gespeichert.")

    return user_list

In [8]:
# Beispielaufruf:
user_ids = scrape_all_films("kritiken_bereinigt_und_gemerged2.csv")


Bereits im Cache gespeicherte User: 715
Film 10053 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10055 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 100673 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10070 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 100714 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 1008 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10080 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10092 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10126 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 101286 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10141 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10220 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 102319 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 102382 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10251 User-IDs aus Cache geladen.
Gesamt User bisher: 715
Film 10257 

In [9]:
import json

# Dateiname des Caches
cache_file = "user_reviews_cache_selenium.json"

# Cache laden
with open(cache_file, encoding="utf-8") as f:
    cache = json.load(f)

# Alle User-IDs aus allen Filmen sammeln
all_user_ids = set()
for user_list in cache.values():
    all_user_ids.update(user_list)

# Ausgabe
print(f"Anzahl Filme im Cache: {len(cache)}")
print(f"Anzahl eindeutiger User-IDs: {len(all_user_ids)}")


Anzahl Filme im Cache: 9367
Anzahl eindeutiger User-IDs: 715
